In [1]:
import re
from rouge import Rouge
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text

def extract_keywords(pre):
    stop_words = set(stopwords.words("english"))
    sentences = sent_tokenize(pre)
    keywords = []
    for sentence in sentences:
        words = word_tokenize(sentence.lower())
        words = [word for word in words if word.isalpha()]
        words = [word for word in words if word not in stop_words]
        keywords.extend(words)
    keywords = nltk.FreqDist(keywords)
    keywords = keywords.most_common(200)
    return [word[0] for word in keywords]

def rank_sentences(sentences, keywords):
    sentence_scores = {}
    for sentence in sentences:
        score = 0
        for keyword in keywords:
            if keyword in sentence.lower():
                score += 1
        sentence_scores[sentence] = score
    sorted_sentences = sorted(sentence_scores.items(), key=lambda x: x[1], reverse=True)
    return [sentence[0] for sentence in sorted_sentences]

def generate_summary(ranked_sentences, model, tokenizer):
    text = " ".join(ranked_sentences)
    input_tokenized = tokenizer.encode(text, return_tensors='pt', max_length=1024, truncation=True)
    summary_ids = model.generate(input_tokenized,
                                  num_beams=9,
                                  no_repeat_ngram_size=3,
                                  length_penalty=2.0,
                                  min_length=150,
                                  max_length=500,
                                  early_stopping=True)
    summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
    return summary

nltk.download('punkt')

df = pd.read_csv("new_summaries.csv")
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")  
model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")

for index, row in df.iterrows():
    text = preprocess_text(row['Text'])
    keywords = extract_keywords(text)
    sentences = sent_tokenize(text)
    ranked_sentences = rank_sentences(sentences, keywords)
    summary = generate_summary(ranked_sentences, model, tokenizer)
    df.at[index, 'Predicted Summary'] = summary

df.to_csv("Output_DistilBart_Sent_Ranking_Keywords_100doc.csv", index=False)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Pranav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import pandas as pd
from rouge import Rouge

# Load the new_summaries.csv file
df = pd.read_csv('Output_DistilBart_Sent_Ranking_Keywords_100doc.csv')

# Create the Rouge object
rouge = Rouge()

# Loop over each row in the dataframe
for index, row in df.iterrows():
    original_summary = row['Original Summary']
    predicted_summary = row['Predicted Summary']
    
    # Calculate the Rouge scores
    scores = rouge.get_scores(predicted_summary, original_summary)
    
    # Extract the R1, R2, and Rl scores
    rouge_1_f1 = scores[0]['rouge-1']['f']
    rouge_2_f1 = scores[0]['rouge-2']['f']
    rouge_l_f1 = scores[0]['rouge-l']['f']
    
    # Update the corresponding columns in the dataframe
    df.at[index, 'R1'] = rouge_1_f1
    df.at[index, 'R2'] = rouge_2_f1
    df.at[index, 'Rl'] = rouge_l_f1

# Save the updated dataframe to a new csv file
df.to_csv("Output_DistilBart_Sent_Ranking_Keywords_100doc.csv", index=False)

In [3]:
# Calculate the average scores
r1_avg = df['R1'].mean()
r2_avg = df['R2'].mean()
rl_avg = df['Rl'].mean()

# Print the average scores
print(f"Average R1 Score: {r1_avg}")
print(f"Average R2 Score: {r2_avg}")
print(f"Average Rl Score: {rl_avg}")

Average R1 Score: 0.2122252202232059
Average R2 Score: 0.06875776352736505
Average Rl Score: 0.18211913385938772


In [4]:
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.metrics import precision, recall, f_measure

df = pd.read_csv('Output_DistilBart_Sent_Ranking_Keywords_100doc.csv')

for index, row in df.iterrows():
    predicted_summary = row['Predicted Summary']
    original_summary = row['Original Summary']
    
    predicted_tokens = set(word_tokenize(predicted_summary))
    original_tokens = set(word_tokenize(original_summary))

    precision_score = precision(original_tokens, predicted_tokens)
    recall_score = recall(original_tokens, predicted_tokens)
    f1_score = f_measure(original_tokens, predicted_tokens)
    
    df.at[index, 'Precision'] = precision_score
    df.at[index, 'Recall'] = recall_score
    df.at[index, 'F1 Score'] = f1_score

df.to_csv('Output_DistilBart_Sent_Ranking_Keywords_100doc.csv', index=False)


In [5]:
import pandas as pd

df = pd.read_csv('Output_DistilBart_Sent_Ranking_Keywords_100doc.csv')

precision_mean = df['Precision'].mean()
recall_mean = df['Recall'].mean()
f1_score_mean = df['F1 Score'].mean()

print("Precision mean:", precision_mean)
print("Recall mean:", recall_mean)
print("F1 Score mean:", f1_score_mean)


Precision mean: 0.41596766970521615
Recall mean: 0.16860821055047115
F1 Score mean: 0.22388654221178084


In [6]:
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu

df = pd.read_csv('Output_DistilBart_Sent_Ranking_Keywords_100doc.csv')

for index, row in df.iterrows():
    predicted_summary = row['Predicted Summary']
    original_summary = row['Original Summary']
    
    predicted_tokens = word_tokenize(predicted_summary)
    original_tokens = word_tokenize(original_summary)

    bleu_score = sentence_bleu([original_tokens], predicted_tokens)
    
    df.at[index, 'BLEU Score'] = bleu_score

df.to_csv('Output_DistilBart_Sent_Ranking_Keywords_100doc.csv', index=False)


import pandas as pd

df = pd.read_csv('Output_DistilBart_Sent_Ranking_Keywords_100doc.csv')

bleu_score_mean = df['BLEU Score'].mean()

print("BLEU Score mean:", bleu_score_mean)

C:\Users\Pranav\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\Pranav\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


BLEU Score mean: 0.0066851032415460735
